# Learner Playground

Notebook created with the intent of offering a handy environment to test out learners and simulations.

#### Import statements
Used to import local functions/modules or external libraries inside the notebook.

In [ ]:
from ola2022_project.environment.environment import example_environment, Step, UserClassParameters
from ola2022_project.simulation.simulation import simulation 
from ola2022_project.learners import AlphalessLearner, AlphaUnitslessLearner

import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# Additional import statements

#### Plotting functions
Functions used to plot data onto a graph for an immediate visualization and analysis.

In [ ]:
# Helper function that computes the means for n slices of a given dimension for an array
def mean_slices(array, dim, n):
    ret = []
    for i in range(n):
        ret.append(np.mean(array[i * dim : dim * (i + 1)]))
    return ret

# Plot colors
colors = ['aqua', 'black', 'blue', 'blueviolet', 'burlywood', 'cadetblue', 'chocolate', 
          'coral', 'crimson', 'cyan', 'forestgreen', 'fuchsia', 'gold', 'green', 'hotpink', 
          'indigo', 'lime', 'magenta','navy', 'orange', 'purple', 'red', 'salmon', 
          'silver', 'teal', 'turquoise', 'violet']
n_colors = len(colors)

In [ ]:
# Function used for quickly plotting multiple experiments onto a graph
def plot_experiments_days(n_days, rewards, days_step=1, show_mean=True, 
                          cumulate=False, show_markers=True, names=None):
    # Calculating and shaping data that is going to be shown on the graph
    days = np.arange(1, n_days + 1, days_step)
    n_exp = len(rewards)
    new_n_days = len(days)
    plot_rewards = np.array(list(map(lambda r: mean_slices(r, days_step, new_n_days), rewards)))
    if(cumulate):
        plot_rewards = np.array(list(map(lambda r: np.cumsum(r), plot_rewards)))
    # Graphical tuning
    marker = "." if show_markers else ""
    plot_colors = \
            random.sample(colors, n_exp) if n_exp <= n_colors else random.choice(colors, n_exp)
    plot_names = ["Experiment {n}" for i in range(len(plot_rewards))]
    if names and len(names) >= len(plot_rewards):
        plot_names = names
    # Creating a new figure and plotting the data onto it
    plt.figure()
    for i, reward in enumerate(plot_rewards):
        plt.plot(days, reward, label=plot_names[i].format(n=i+1), marker=marker, 
                 linestyle='-', color=plot_colors[i])
        if(show_mean):
            mean = [np.mean(plot_rewards[i])] * new_n_days
            plt.plot(days, mean, label="Mean of " + plot_names[i].format(n=i+1), 
                     linestyle='--', color=plot_colors[i])
    # Setting labels and showing the figure
    plt.xlabel("days")
    plt.ylabel("reward")
    plt.legend(loc = "best")
    plt.show()

In [ ]:
# Additional plotting functions

#### Environment setup
Create the environment reference with customizable parameters.

In [ ]:
# Random Number Generator used as a source of randomness by the environment and the simulation
rng = np.random.default_rng()

# Environment separated default parameters
env_classes_parameters = [
    [
        UserClassParameters(8, 0.06, 1, 160),
        UserClassParameters(10, 0.06, 1, 150),
        UserClassParameters(10, 0.07, 0.8, 160),
        UserClassParameters(8, 0.03, 0.4, 120),
        UserClassParameters(7, 0.06, 0.7, 150),
        UserClassParameters(14, 0.08, 0.1, 180),
    ],
    [
        UserClassParameters(25, 0.06, 1, 120),
        UserClassParameters(22, 0.03, 0.4, 100),
        UserClassParameters(20, 0.04, 0.5, 80),
        UserClassParameters(16, 0.05, 0.4, 150),
        UserClassParameters(24, 0.07, 0.3, 100),
        UserClassParameters(20, 0.03, 0.1, 110),
    ],
    [
        UserClassParameters(26, 0.03, 0.5, 120),
        UserClassParameters(33, 0.08, 0.5, 260),
        UserClassParameters(25, 0.07, 1.0, 280),
        UserClassParameters(30, 0.06, 0.6, 250),
        UserClassParameters(31, 0.03, 0.2, 290),
        UserClassParameters(36, 0.07, 0.3, 260),
    ],
]

In [ ]:
# Creating the environment
env = example_environment(
    total_budget=300,
    class_ratios=[0.3, 0.6, 0.1],
    product_prices=[3, 15, 8, 22, 1],
    classes_parameters=env_classes_parameters,
    competitor_budget=100,
    lam=0.5,
    max_items=3,
    graph_fully_connected=True,
    graph_zeros_probability=0.5,
    next_products=[(2, 3), (0, 2), (1, 4), (4, 1), (3, 0)],
)

In [ ]:
# Additional environment tuning

#### Running the simulation
Execute the simulation and collect the results.

In [ ]:
# Simulation separated default parameters
sim_n_days=50

In [ ]:
# Running the simulation
simulation_rewards, foo = simulation(
    rng,
    env,
    learner_factory=AlphalessLearner,
    n_customers_mean=100,
    n_customers_variance=5,
    n_experiment=3,
    n_days=sim_n_days,
    n_budget_steps=20,
    step=Step.ONE
)

In [ ]:
# Additional execution code

#### Plotting the results
Once the simulation has been run, collect the results and display them nicely using plotting functions.

In [ ]:
# Plotting the experiment
plot_experiments_days(sim_n_days, simulation_rewards, days_step=1, 
                      show_mean=True, cumulate=False, show_markers=True, 
                      names=["Experiment {n}"])

In [ ]:
# Additional plotting